In [1]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm_notebook
import bs4
import re
import os
from requests.exceptions import ConnectionError
from urllib3.exceptions import ProtocolError
from http.client import RemoteDisconnected
import time

In [2]:
%load_ext autoreload
%autoreload 2

from sofa_score_scrap import *

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
}

#### Dê/Para de Siglas

In [34]:
de_para_siglas = pd.read_excel("de_para_siglas_br.xlsx")
de_para_siglas["time"] = de_para_siglas["time"].astype(str)
de_para_siglas["time"] = de_para_siglas["time"].apply(str.strip)
de_para_siglas.set_index("time", inplace=True)

### Get Incidentes

In [4]:
resp = requests.get('https://api.sofascore.com/api/v1/event/8684164/incidents', headers=headers)

In [6]:
resp.json()

{'incidents': [{'text': 'FT',
   'homeScore': 2,
   'awayScore': 1,
   'isLive': False,
   'time': 90,
   'addedTime': 999,
   'incidentType': 'period'},
  {'playerIn': {'name': 'Billy Gilmour',
    'slug': 'gilmour-billy',
    'shortName': 'B. Gilmour',
    'position': 'M',
    'userCount': 2700,
    'id': 907668},
   'playerOut': {'name': 'Christian Pulišić',
    'slug': 'christian-pulisic',
    'shortName': 'C. Pulišić',
    'position': 'M',
    'userCount': 20255,
    'id': 817957},
   'id': 119665608,
   'time': 90,
   'addedTime': 1,
   'isHome': True,
   'incidentType': 'substitution'},
  {'playerIn': {'name': 'Pedro',
    'slug': 'pedro',
    'shortName': 'Pedro',
    'position': 'F',
    'userCount': 5316,
    'id': 35428},
   'playerOut': {'name': 'Mason Mount',
    'slug': 'mason-mount',
    'shortName': 'M. Mount',
    'position': 'M',
    'userCount': 11328,
    'id': 836694},
   'id': 119665607,
   'time': 90,
   'addedTime': 1,
   'isHome': True,
   'incidentType': 'subs

In [13]:
incidents_per_type = {}

for el in resp.json()["incidents"]:
    if not el['incidentType'] in incidents_per_type:
        incidents_per_type[el['incidentType']] = []
    incidents_per_type[el['incidentType']].append(el)

In [17]:
for key in incidents_per_type:
    incidents_per_type[key] = pd.DataFrame(incidents_per_type[key])

In [8]:
df_incidents = pd.DataFrame(resp.json()["incidents"])

In [11]:
df_incidents[df_incidents.incidentType == 'goal']

,text,homeScore,awayScore,isLive,time,addedTime,incidentType,playerIn,playerOut,id,isHome,length,player,incidentClass,playerName,reason,confirmed
4,NaN,2.0,1.0,NaN,78,NaN,goal,NaN,NaN,130804277.0,True,NaN,"{'name': 'Willian', 'slug': 'willian', 'shortN...",penalty,NaN,NaN,NaN
13,NaN,1.0,1.0,NaN,55,NaN,goal,NaN,NaN,130804278.0,False,NaN,"{'name': 'Kevin De Bruyne', 'slug': 'kevin-de-...",regular,NaN,NaN,NaN
19,NaN,1.0,0.0,NaN,36,NaN,goal,NaN,NaN,130804276.0,True,NaN,"{'name': 'Christian Pulišić', 'slug': 'christi...",regular,NaN,NaN,NaN


In [10]:
df_incidents.incidentType.value_counts()

substitution    8
card            3
goal            3
period          2
varDecision     2
injuryTime      2
Name: incidentType, dtype: int64

### Players Stats 

In [29]:
# s.get('https://www.sofascore.cddom/pt/atletico-mineiro-internacional/qOsCO', headers={'x-test2': 'true'})
resp = requests.get("https://www.sofascore.com/event/7724681/statistics/players/json?_=154328071", headers=headers)

In [30]:
players_df = pd.DataFrame(resp.json()["players"])

In [35]:
get_per_player_data(players_df, de_para_siglas)

,name,slug,shortName,id,disabled,national,team_name,team_id,goals,goalAssist,...,totalLongBalls,possessionLost,fouls,wasFouled,offsides,saves,punches,runsOut,goodHighClaim,team
game,,,,,,,,,,,,,,,,,,,,,
INT X CAM 2018-11-21,Juan Cazares,atletico-mineiro,J. Cazares,158411,False,False,Atlético Mineiro,1977,1,1,...,2 (2),17,2,1,NaN,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Andres D'Alessandro,internacional,A. D'Alessandro,17639,NaN,False,Internacional,1966,1,,...,5 (2),23,1,3,NaN,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,Víctor Cuesta,internacional,V. Cuesta,587148,NaN,False,Internacional,1966,,,...,11 (7),13,2,3,NaN,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,David Terans,atletico-mineiro,D. Terans,586662,False,False,Atlético Mineiro,1977,1,,...,2 (1),5,1,0,NaN,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Nico López,internacional,N. López,255881,NaN,False,Internacional,1966,,,...,7 (4),15,1,1,NaN,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,Patric,atletico-mineiro,Patric,78853,False,False,Atlético Mineiro,1977,,,...,8 (1),22,1,1,NaN,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Luan,atletico-mineiro,Luan,243427,False,False,Atlético Mineiro,1977,,,...,1 (0),8,4,1,NaN,NaN,NaN,NaN,NaN,CAM
INT X CAM 2018-11-21,Emerson Santos,internacional,E. Santos,844015,NaN,False,Internacional,1966,,,...,4 (2),9,0,0,NaN,NaN,NaN,NaN,NaN,INT
INT X CAM 2018-11-21,Fabiano,internacional,F. Leismann,551304,NaN,False,Internacional,1966,,,...,5 (2),20,2,0,NaN,NaN,NaN,NaN,NaN,INT


### Game Stats

In [43]:
resp = requests.get("https://www.sofascore.com/event/7724681/general/json", headers=headers)

In [38]:
pd.DataFrame(game_statistics(resp, players_df, de_para_siglas, period=0), index=[0]).set_index("game")

,team_away,team_home,data,hora,away_score,home_score,result,Ball possession_home,Ball possession_away,Total shots_home,...,form_minute_82,form_minute_83,form_minute_84,form_minute_85,form_minute_86,form_minute_87,form_minute_88,form_minute_89,form_minute_90,form_minute_90.5
game,,,,,,,,,,,,,,,,,,,,,
INT X CAM 2018-11-21,CAM,INT,2018-11-21,19:30:00,2,1,-1,54%,46%,15,...,22,73,41,24,1,1,63,29,28,-49


In [48]:
resp.json()["statistics"]["periods"][0]['period']

'ALL'

### Pega links dos jogos históricos

#### Pega os IDs das temporadas

In [111]:
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/brazil/brasileiro-serie-a/325", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/england/premier-league/17", headers=headers)
# comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/spain/laliga/8", headers=headers)
comp = requests.get("https://www.sofascore.com/pt/torneio/futebol/germany/bundesliga/35", headers=headers)
c = bs4.BeautifulSoup(comp.content)

In [112]:
temporadas = c.find_all("script", attrs={"id":"__NEXT_DATA__"})

In [113]:
temp = str(temporadas[0]).replace('<script id="__NEXT_DATA__" type="application/json">', '')
temp = temp.replace('</script>', '')

IndexError: list index out of range

In [110]:
temp_j = json.loads(temp)

In [104]:
season_ids = temp_j['props']['initialProps']['pageProps']['seasons']

In [105]:
season_ids

[{'name': 'LaLiga 19/20', 'year': '19/20', 'id': 24127},
 {'name': 'LaLiga 18/19', 'year': '18/19', 'id': 18020},
 {'name': 'LaLiga 17/18', 'year': '17/18', 'id': 13662},
 {'name': 'LaLiga 16/17', 'year': '16/17', 'id': 11906},
 {'name': 'Primera Division 15/16', 'year': '15/16', 'id': 10495},
 {'name': 'Primera Division 14/15', 'year': '14/15', 'id': 8578},
 {'name': 'Primera Division 13/14', 'year': '13/14', 'id': 6559},
 {'name': 'Primera Division 12/13', 'year': '12/13', 'id': 4959},
 {'name': 'Primera Division 11/12', 'year': '11/12', 'id': 3502},
 {'name': 'Liga BBVA 10/11', 'year': '10/11', 'id': 2896},
 {'name': 'Liga BBVA 09/10', 'year': '09/10', 'id': 2252},
 {'name': 'Liga BBVA 08/09', 'year': '08/09', 'id': 1587},
 {'name': 'Primera Division 07/08', 'year': '07/08', 'id': 669},
 {'name': 'Primera Division 06/07', 'year': '06/07', 'id': 102},
 {'name': 'Primera Division 05/06', 'year': '05/06', 'id': 101},
 {'name': 'Primera Division 04/05', 'year': '04/05', 'id': 100},
 {'n

In [106]:
json.dump(season_ids, open("IDs_La_Liga_seasons.json", 'w'))

In [52]:
# temporadas = c.find_all("a", attrs={"class":"pointer js-uniqueTournament-page-load-season"})

id_temporada = [(a.text.strip(), a.attrs["data-season-id"]) for a in temporadas]
id_temporada

[]

In [24]:
r = resp.json()

In [23]:
r = bs4.BeautifulSoup(resp.text)

In [25]:
teams = set()

for x in r["roundMatches"]["tournaments"]:
    for y in x["events"]:
        teams.add(y["homeTeam"]["name"])
        teams.add(y["awayTeam"]["name"])

teams

KeyError: 'roundMatches'

## Pega Links dos jogos por Rodadas

#### Página Inicial

In [82]:
s = requests.Session()
s.get(comp.url, headers=headers)

<Response [200]>

#### Para cada temporada

In [84]:
id_temporada[:17]

[]

In [116]:
games = resp.json()['roundMatches']["tournaments"][0]['events']

In [121]:
for temp in id_temporada[:17]:
    ret = []
    
    for rodada in tqdm_notebook(range(1, 39)):
        # print(temp[0] + ": " + str(rodada), end="\r")
        resp = s.get("https://www.sofascore.com/u-tournament/17/season/"+ temp[1] + "/matches/round/" + str(rodada), headers=headers)
        resp.encoding = 'UTF-8'
        ret.append(get_info_rodada(resp, de_para_siglas))
    
    ret = pd.concat(ret)
    ret.to_csv("links_sofa_score/sofa_score_links_EPL_"+temp[0].replace("/", "_")+".csv", sep=";")

### Pega informações dos jogos históricos

In [7]:
base = pd.read_csv("links_sofa_score/sofa_score_links_EPL_18_19.csv", sep=";")

In [8]:
list(reversed(sorted(os.listdir("links_sofa_score/"))))[1:]

['sofa_score_links_EPL_18_19.csv',
 'sofa_score_links_EPL_17_18.csv',
 'sofa_score_links_EPL_16_17.csv',
 'sofa_score_links_EPL_15_16.csv',
 'sofa_score_links_EPL_14_15.csv',
 'sofa_score_links_EPL_13_14.csv',
 'sofa_score_links_EPL_12_13.csv',
 'sofa_score_links_EPL_11_12.csv',
 'sofa_score_links_EPL_10_11.csv',
 'sofa_score_links_EPL_09_10.csv',
 'sofa_score_links_EPL_08_09.csv',
 'sofa_score_links_EPL_07_08.csv',
 'sofa_score_links_EPL_06_07.csv',
 'sofa_score_links_EPL_05_06.csv',
 'sofa_score_links_EPL_04_05.csv',
 'sofa_score_links_EPL_03_04.csv',
 'sofa_score_links_BR_2018.csv',
 'sofa_score_links_BR_2017.csv',
 'sofa_score_links_BR_2016.csv',
 'sofa_score_links_BR_2015.csv',
 'sofa_score_links_BR_2014.csv',
 'sofa_score_links_BR_2013.csv',
 'sofa_score_links_BR_2012.csv',
 'sofa_score_links_BR_2011.csv',
 'sofa_score_links_BR_2010.csv',
 'sofa_score_links_BR_2009.csv',
 'sofa_score_links_BR_2008.csv']

In [35]:
de_para_siglas = pd.read_excel("de_para_siglas_epl.xlsx")
de_para_siglas["time"] = de_para_siglas["time"].astype(str)
de_para_siglas["time"] = de_para_siglas["time"].apply(str.strip)
de_para_siglas.set_index("time", inplace=True)

In [37]:
erros = []

for file in list(reversed(sorted(os.listdir("links_sofa_score/"))))[5:]:
    base = pd.read_csv("links_sofa_score/" + file, sep=";")
    
    base[(base.home_score != -1) & (base.away_score != -1)]
    
    resp_game = []
    resp_players = []
    game_id = 0
    
    name = file[-13:-4]
    
    i = 0
    tentativas = 3
    while(tentativas > 0):
        try:
            while (game_id < len(base.id)):
                print(name + " Game #" + str(i), end="\r")
                resp = s.get("https://www.sofascore.com/event/" + str(base.id.iloc[game_id]) + "/general/json",
                             headers=headers, timeout=10)

                resp2 = s.get("https://www.sofascore.com/event/" + str(base.id.iloc[game_id]) + "/statistics/players/json",
                              headers=headers, timeout=10)
                resp2.encoding = 'UTF-8'
                
                if "Statistics not found." in str(resp2.content):
                    erros.append(base.id.iloc[game_id])
                    tentativas = 0
                    game_id += 1
                    continue
                    
                try:
                    players_df = pd.DataFrame(resp2.json()["players"])     
                except KeyError:
                    resp_game.append(pd.DataFrame(game_statistics(resp, players_df, de_para_siglas), index=[0]).set_index("game"))
                    tentativas = 0
                    game_id += 1
                    continue
                    
                if players_df.empty:
                    tentativas = 0
                    game_id += 1
                    continue

                resp_game.append(
                    pd.DataFrame(game_statistics(resp, players_df, de_para_siglas), index=[0]).set_index("game")
                )
                resp_players.append(get_per_player_data(players_df, de_para_siglas)) 
                i = i + 1
                
                tentativas = 0
                game_id += 1
        except (ConnectionError, ProtocolError, RemoteDisconnected):
            print("Error")
            time.sleep(2)
            tentativas -= 1

    ret_game = pd.concat(resp_game)
    ret_game.to_csv("EPL_data_sofa_score/game_data_"+ name +".csv", sep=";")
    
    ret_players = pd.concat(resp_players)
    ret_players.to_csv("EPL_data_sofa_score/players_data_"+ name +".csv", sep=";")
    
    resp_game = []
    resp_players = []
    game_id = 0

/home/rafael/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



TypeError: 'NoneType' object is not subscriptable